<a href="https://colab.research.google.com/github/kitsamho/imdb_app/blob/main/Simple_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! pip install transformers==3.0.2

     |████████████████████████████████| 769 kB 8.4 MB/s 
     |████████████████████████████████| 1.2 MB 30.8 MB/s 
     |████████████████████████████████| 3.0 MB 62.3 MB/s 
     |████████████████████████████████| 895 kB 61.4 MB/s 


In [1]:
# Importing stock ml libraries
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)

2021-08-01 13:59:55.702263: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_json('/content/drive/MyDrive/tmdb_data_main.json')

In [4]:
df = df[['tmdb_id','movie','overview','genres','genres_new','popularity','release_year','vote_average','budget','revenue','poster_path']]
df = df.dropna(subset=['genres'])
genre_include = df.explode('genres').genres.value_counts().keys()[0:12]

def genres_check(film_genres):
    
    for genre in film_genres:
        if genre not in genre_include:
            return False
    return True

def multi_label_prep(film_genres):
    prep = []
    for genre in genre_include:
        if genre in film_genres:
            prep.append(1)
        else:
            prep.append(0)
            
    return prep

df['genres_valid'] = df['genres'].apply(genres_check)
df = df[df.genres_valid ==True]
df.shape
genre_include = list(genre_include)
genre_include.sort()
genre_include



df['labels'] = df['genres'].apply(multi_label_prep)

In [5]:
df = pd.concat([df[['overview']],df[['labels']]['labels'].apply(pd.Series)],axis=1)

In [6]:
df.columns = ['overview']+genre_include
df

,overview,Action,Adventure,Animation,Comedy,Crime,Drama,Family,Fantasy,Horror,Romance,Science Fiction,Thriller
0,Jack Torrance accepts a caretaker job at the O...,0,0,0,0,0,0,0,0,1,0,0,1
2,"Spanning the years 1945 to 1955, a chronicle o...",0,0,0,0,1,1,0,0,0,0,0,0
4,"In a near-future Britain, young Alexander DeLa...",0,0,0,0,0,1,0,0,0,0,1,0
5,Camp counselors are stalked and murdered by an...,0,0,0,0,0,0,0,0,1,0,0,0
6,A Coca-Cola bottle dropped from an airplane ra...,1,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19615,The story of Ba Sang – the second of the 4 noi...,0,0,0,1,0,1,1,0,0,0,0,0
19622,Drill-bit is a drilling prodigy who is offered...,0,1,0,0,1,0,0,0,0,0,0,0
19638,"Bob Spit, a comic book character, lives in a p...",0,0,1,0,0,0,0,0,0,0,0,0
19639,"Following the end of the acclaimed tv series, ...",0,1,0,1,0,0,0,0,0,0,0,0


In [7]:
from sklearn.model_selection import train_test_split


train_df, eval_df = train_test_split(df, test_size=0.2)

In [8]:
train_df

,overview,Action,Adventure,Animation,Comedy,Crime,Drama,Family,Fantasy,Horror,Romance,Science Fiction,Thriller
18621,"A true story of Samy, native of La Courneuve, ...",0,1,0,1,0,0,0,0,0,1,0,0
16591,Leo Borlock is an average student at Mica High...,0,0,0,1,0,1,0,0,0,1,0,0
16956,"In pre-war Italy, a young couple have a baby b...",0,0,0,0,0,1,0,0,0,0,0,0
2202,A kindly occupational therapist undergoes a ne...,0,0,0,1,0,1,0,0,0,0,1,0
5624,"Favoring pedigree dogs, a new regulation puts ...",0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3172,"In the future, medical technology has advanced...",1,0,0,0,1,0,0,0,0,0,1,1
13332,"Faust inhabits an earthy, 19th-century world o...",0,0,0,0,0,1,0,1,0,0,0,0
16493,"In the 1960s, two entrepreneurs hatch an ingen...",0,0,0,0,0,1,0,0,0,0,0,0
14199,"Loving girlfriend, family fortune, breakout mo...",0,0,0,0,0,1,0,0,0,0,1,0


In [12]:
!pip install simpletransformers

     |████████████████████████████████| 221 kB 8.3 MB/s 
     |████████████████████████████████| 2.6 MB 41.3 MB/s 
     |████████████████████████████████| 76 kB 5.6 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 7.9 MB 54.9 MB/s 
     |████████████████████████████████| 264 kB 53.3 MB/s 
     |████████████████████████████████| 1.7 MB 59.1 MB/s 
     |████████████████████████████████| 124 kB 75.8 MB/s 
     |████████████████████████████████| 636 kB 55.7 MB/s 
     |████████████████████████████████| 3.3 MB 55.3 MB/s 
     |████████████████████████████████| 133 kB 74.4 MB/s 
     |████████████████████████████████| 170 kB 67.1 MB/s 
     |████████████████████████████████| 188 kB 76.6 MB/s 
     |████████████████████████████████| 63 kB 2.6 MB/s 
     |████████████████████████████████| 118 kB 68.4 MB/s 
     |████████████████████████████████| 243 kB 67.1 MB/s 
     |████████████████████████████████| 75 kB 5.1 MB/s 
     |█████████████████

In [9]:
from simpletransformers.classification import MultiLabelClassificationModel

In [10]:
import torch


cuda_available = torch.cuda.is_available()



In [11]:
model = MultiLabelClassificationModel('distilbert', 'distilbert-base-uncased', num_labels=12, args={'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 1, 'max_seq_length': 512},use_cuda=True)

Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForMultiLabelSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForMultiLabelSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.wei

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [12]:
model.train_model(train_df)

  0%|          | 0/12000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/6000 [00:00<?, ?it/s]

RuntimeError: ignored